In [1]:
huespedes = []
habitaciones = [
    {"numero": 101, "tipo": "Estándar", "disponible": True},
    {"numero": 102, "tipo": "Estándar", "disponible": True},
    {"numero": 201, "tipo": "Suite", "disponible": True},
    {"numero": 202, "tipo": "Suite", "disponible": True}
]
reservas = []

# MENÚ PRINCIPAL
def menu():
    while True:
        print("\n========= HOTEL PALMERAS =========")
        print("1. Registrar Huésped")
        print("2. Realizar Reserva")
        print("3. Realizar Check-In (día actual)")
        print("4. Registrar Salida (Check-Out)")
        print("5. Acceso Administrador")
        print("6. Ver Reportes Gráficos")
        print("7. Salir")
        opcion = input("Seleccione una opción (1-7): ")

        if opcion == "1":
            registrar_huesped()
        elif opcion == "2":
            reservar_habitacion(huespedes)
        elif opcion == "3":
            check_in(huespedes, reservas)
        elif opcion == "4":
            doc = input("Documento del huésped: ")
            check_out(doc, huespedes, reservas)
        elif opcion == "5":
            ruta = "admin.txt"
            df_admin = cargar_usuarios_admin(ruta)
            if login_admin(df_admin):
                reporte_estadistico(huespedes, reservas, habitaciones)
        elif opcion == "6":
            generar_graficas(habitaciones, reservas)
        elif opcion == "7":
            print("Gracias por usar el sistema. ¡Hasta luego!")
            break
        else:
            print("Opción inválida. Intente de nuevo.")


In [2]:
# Registro de Huéspedes

import re

huespedes = []

def validar_nombre_apellido(campo):
    errores = []
    if len(campo) < 3:
        errores.append("Debe tener al menos 3 caracteres.")
    if not campo.isalpha():
        errores.append("Solo puede contener letras.")
    return errores

def validar_documento(doc):
    errores = []
    if not doc.isdigit():
        errores.append("Solo puede contener números.")
    if len(doc) < 3 or len(doc) > 15:
        errores.append("Debe tener entre 3 y 15 dígitos.")
    return errores

def validar_correo(correo):
    if re.match(r"[^@]+@[^@]+\.[^@]+", correo):
        return []
    return ["Correo electrónico inválido."]

def validar_telefono(tel):
    errores = []
    if not tel.isdigit():
        errores.append("Solo puede contener números.")
    if len(tel) < 7 or len(tel) > 15:
        errores.append("Debe tener entre 7 y 15 dígitos.")
    return errores

def registrar_huesped():
    print("\n--- Registro de Huésped ---")
    nombre = input("Nombre: ").strip()
    apellido = input("Apellido: ").strip()
    documento = input("Documento de identidad: ").strip()
    correo = input("Correo electrónico: ").strip()
    telefono = input("Teléfono de contacto: ").strip()

    errores = {
        "nombre": validar_nombre_apellido(nombre),
        "apellido": validar_nombre_apellido(apellido),
        "documento": validar_documento(documento),
        "correo": validar_correo(correo),
        "telefono": validar_telefono(telefono)
    }

    errores_totales = [f"{campo}: {', '.join(msgs)}" for campo, msgs in errores.items() if msgs]
    if errores_totales:
        print("\n Errores encontrados:")
        for e in errores_totales:
            print("-", e)
    else:
        huespedes.append({
            "nombre": nombre,
            "apellido": apellido,
            "documento": documento,
            "correo": correo,
            "telefono": telefono
        })
        print(" Huésped registrado exitosamente.")


In [3]:
# Reservas de Hotel

from datetime import datetime, timedelta

# Simulación de base de datos
habitaciones = [
    {"numero": 101, "tipo": "Estándar", "disponible": True},
    {"numero": 102, "tipo": "Estándar", "disponible": True},
    {"numero": 201, "tipo": "Suite", "disponible": True},
    {"numero": 202, "tipo": "Suite", "disponible": True}
]

reservas = []

def buscar_huesped(documento, lista_huespedes):
    for h in lista_huespedes:
        if h["documento"] == documento:
            return h
    return None

def mostrar_habitaciones_disponibles(tipo):
    return [h for h in habitaciones if h["tipo"] == tipo and h["disponible"]]

def reservar_habitacion(lista_huespedes):
    print("\n--- Realizar Reserva ---")
    doc = input("Ingrese documento del huésped: ").strip()
    huesped = buscar_huesped(doc, lista_huespedes)

    if not huesped:
        print("Huésped no registrado.")
        return

    tipo = input("Tipo de habitación (Estándar/Suite): ").capitalize()
    if tipo not in ["Estándar", "Suite"]:
        print("Tipo de habitación inválido.")
        return

    disponibles = mostrar_habitaciones_disponibles(tipo)
    if not disponibles:
        print(f"No hay habitaciones {tipo} disponibles.")
        return

    try:
        fecha_ingreso_str = input("Fecha de ingreso (AAAA-MM-DD): ")
        fecha_ingreso = datetime.strptime(fecha_ingreso_str, "%Y-%m-%d")
        noches = int(input("Número de noches de estancia: "))
        if noches <= 0:
            print("Las noches deben ser al menos 1.")
            return
    except:
        print("Fecha o noches inválidas.")
        return

    fecha_salida = fecha_ingreso + timedelta(days=noches)
    habitacion = disponibles[0]  # Tomamos la primera disponible
    habitacion["disponible"] = False  # La marcamos como ocupada

    precio_noche = 120000 if tipo == "Estándar" else 250000
    total = precio_noche * noches

    reserva = {
        "documento": doc,
        "nombre": huesped["nombre"],
        "tipo_habitacion": tipo,
        "numero_habitacion": habitacion["numero"],
        "fecha_ingreso": fecha_ingreso_str,
        "fecha_salida": fecha_salida.strftime("%Y-%m-%d"),
        "noches": noches,
        "total_estimado": total
    }

    reservas.append(reserva)

    print("\n Reserva confirmada:")
    print(f"Nombre: {huesped['nombre']} {huesped['apellido']}")
    print(f"Habitación: {tipo} #{habitacion['numero']}")
    print(f"Ingreso: {fecha_ingreso_str}")
    print(f"Salida: {fecha_salida.strftime('%Y-%m-%d')}")
    print(f"Noches: {noches}")
    print(f"Costo estimado: ${total:,.0f}")


In [4]:
# Check-In Hotel

from datetime import datetime

def check_in(huespedes, reservas):
    print("\n--- Check-In Real ---")
    doc = input("Documento del huésped: ").strip()
    huesped = buscar_huesped(doc, huespedes)
    if not huesped:
        print("Huésped no registrado.")
        return

    hoy = datetime.today().strftime("%Y-%m-%d")
    reserva = next((r for r in reservas if r["documento"] == doc and r["fecha_ingreso"] == hoy), None)

    if not reserva:
        print("No hay reservas activas para hoy.")
        return

    print(f"Check-In confirmado para {huesped['nombre']} habitación {reserva['tipo_habitacion']} #{reserva['numero_habitacion']}")


In [5]:
# Check-Out Hotel

def check_out(documento, lista_huespedes, lista_reservas):
    print("\n--- Registro de Salida ---")
    huesped = buscar_huesped(documento, lista_huespedes)

    if not huesped:
        print("Huésped no registrado.")
        return

    # Buscar reserva activa
    reserva = next((r for r in lista_reservas if r["documento"] == documento), None)

    if not reserva:
        print("No hay una reserva activa para este huésped.")
        return

    # Calcular costo total
    noches = reserva["noches"]
    tipo = reserva["tipo_habitacion"]
    precio_noche = 120000 if tipo == "Estándar" else 250000
    total = max(precio_noche, noches * precio_noche)

    # Generar factura
    print("\n Factura de Estancia:")
    print(f"Nombre: {huesped['nombre']} {huesped['apellido']}")
    print(f"Documento: {huesped['documento']}")
    print(f"Habitación: {tipo} #{reserva['numero_habitacion']}")
    print(f"Ingreso: {reserva['fecha_ingreso']}")
    print(f"Salida: {reserva['fecha_salida']}")
    print(f"Noches: {noches}")
    print(f"Total a pagar: ${total:,.0f}")

    # Eliminar reserva y liberar habitación
    lista_reservas.remove(reserva)
    for h in habitaciones:
        if h["numero"] == reserva["numero_habitacion"]:
            h["disponible"] = True
            break


In [6]:
# Administración

import pandas as pd

def cargar_usuarios_admin(ruta):
    try:
        df = pd.read_csv(ruta, header=None, names=["usuario", "clave"])
        return df
    except:
        print("Error al cargar archivo de usuarios.")
        return pd.DataFrame()

def login_admin(df_admins):
    print("\n--- Inicio de Sesión Administrador ---")
    user = input("Usuario: ").strip()
    clave = input("Contraseña: ").strip()

    existe = df_admins[
        (df_admins["usuario"] == user) & (df_admins["clave"] == clave)
    ]

    if not existe.empty:
        print("Acceso concedido.")
        return True
    else:
        print("Usuario o contraseña incorrectos.")
        return False

def reporte_estadistico(huespedes, reservas, habitaciones):
    print("\n Reportes Administrativos:")

    total_huespedes = len(huespedes)
    ocupadas = [h for h in habitaciones if not h["disponible"]]
    disponibles = [h for h in habitaciones if h["disponible"]]
    ingresos_totales = sum([r["total_estimado"] for r in reservas])
    total_reservas = len(reservas)
    total_noches = sum([r["noches"] for r in reservas])
    promedio_noches = total_noches / total_reservas if total_reservas > 0 else 0

    print(f"- Total huéspedes registrados: {total_huespedes}")
    print(f"- Habitaciones ocupadas: {len(ocupadas)}")
    print(f"- Habitaciones disponibles: {len(disponibles)}")
    print(f"- Ingresos totales estimados: ${ingresos_totales:,.0f}")
    print(f"- Promedio de noches por huésped: {promedio_noches:.2f}")

    # Historial de huéspedes
    print("\n Historial de reservas:")
    for r in reservas:
        print(f"• {r['nombre']} - {r['tipo_habitacion']} #{r['numero_habitacion']} - {r['noches']} noches")

    # Huésped con más noches
    noches_por_huesped = {}
    for r in reservas:
        doc = r["documento"]
        noches_por_huesped[doc] = noches_por_huesped.get(doc, 0) + r["noches"]

    if noches_por_huesped:
        doc_max = max(noches_por_huesped, key=noches_por_huesped.get)
        doc_min = min(noches_por_huesped, key=noches_por_huesped.get)
        huesped_max = buscar_huesped(doc_max, huespedes)
        huesped_min = buscar_huesped(doc_min, huespedes)

        print(f"\n Huésped con más noches: {huesped_max['nombre']} ({noches_por_huesped[doc_max]} noches)")
        print(f" Huésped con menos noches: {huesped_min['nombre']} ({noches_por_huesped[doc_min]} noches)")
    else:
        print("\n No hay reservas para calcular mayor o menor estancia.")


In [9]:
%%writefile admin.txt
admin1,admin123
admin2,clave456


Writing admin.txt


In [13]:
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime

def generar_graficas(habitaciones, reservas):
    print("\n Generando gráficas...")

    # 1. Barras: habitaciones ocupadas por tipo
    tipos = ["Estándar", "Suite"]
    ocupadas = [h for h in habitaciones if not h["disponible"]]
    contador = Counter(h["tipo"] for h in ocupadas)

    if ocupadas:
        plt.bar(tipos, [contador.get("Estándar", 0), contador.get("Suite", 0)])
        plt.title("Habitaciones ocupadas por tipo")
        plt.xlabel("Tipo")
        plt.ylabel("Cantidad")
        plt.show()
    else:
        print("No hay habitaciones ocupadas para graficar (1).")

    # 2. Pie: ocupadas vs disponibles
    ocupadas_total = len(ocupadas)
    disponibles_total = len([h for h in habitaciones if h["disponible"]])
    if ocupadas_total + disponibles_total > 0:
        plt.pie([ocupadas_total, disponibles_total], labels=["Ocupadas", "Disponibles"], autopct="%1.1f%%")
        plt.title("Distribución de habitaciones")
        plt.show()
    else:
        print("No hay habitaciones cargadas para graficar (2).")

    # 3. Línea: Check-out por día
    fechas = [r["fecha_salida"] for r in reservas]
    if fechas:
        conteo = Counter(fechas)
        fechas_ordenadas = sorted(conteo.keys())
        valores = [conteo[f] for f in fechas_ordenadas]
        plt.plot(fechas_ordenadas, valores, marker='o')
        plt.title("Check-outs por día")
        plt.xlabel("Fecha")
        plt.ylabel("Cantidad")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("No hay fechas de check-out para graficar (3).")

    # 4. Barras horizontales: top 10 huéspedes por noches
    noches_por_doc = Counter()
    for r in reservas:
        noches_por_doc[r["documento"]] += r["noches"]

    if noches_por_doc:
        top10 = noches_por_doc.most_common(10)
        labels = [doc for doc, _ in top10]
        valores = [noches for _, noches in top10]
        plt.barh(labels, valores)
        plt.title("Top 10 huéspedes por noches")
        plt.xlabel("Noches")
        plt.ylabel("Documento")
        plt.tight_layout()
        plt.show()
    else:
        print("No hay datos de noches por huésped para graficar (4).")

    # 5. Dispersión: noches vs total pagado
    noches = [r["noches"] for r in reservas]
    totales = [r["total_estimado"] for r in reservas]

    if noches and totales:
        plt.scatter(noches, totales)
        plt.title("Relación noches vs total pagado")
        plt.xlabel("Noches")
        plt.ylabel("Total pagado")
        plt.show()
    else:
        print("No hay datos suficientes para graficar dispersión (5).")

    # 6. Pie: ingresos por tipo de habitación
    ingresos = {"Estándar": 0, "Suite": 0}
    for r in reservas:
        ingresos[r["tipo_habitacion"]] += r["total_estimado"]

    if sum(ingresos.values()) > 0:
        plt.pie(ingresos.values(), labels=ingresos.keys(), autopct="%1.1f%%")
        plt.title("Ingresos por tipo de habitación")
        plt.show()
    else:
        print("No hay ingresos registrados para graficar (6).")

    # 7. Histograma: duración de estancias
    duraciones = [r["noches"] for r in reservas]
    if duraciones:
        plt.hist(duraciones, bins=range(1, max(duraciones) + 2), edgecolor='black')
        plt.title("Distribución de estancias (noches)")
        plt.xlabel("Noches")
        plt.ylabel("Cantidad")
        plt.show()
    else:
        print("No hay duraciones de estancia para graficar (7).")

    # 8. Combinada: ingresos diarios (barras) + huéspedes diarios (línea)
    ingresos_dia = Counter()
    huespedes_dia = Counter()
    for r in reservas:
        ingresos_dia[r["fecha_ingreso"]] += r["total_estimado"]
        huespedes_dia[r["fecha_ingreso"]] += 1

    fechas = sorted(set(ingresos_dia.keys()) | set(huespedes_dia.keys()))
    if fechas:
        ingresos_vals = [ingresos_dia[f] for f in fechas]
        huespedes_vals = [huespedes_dia[f] for f in fechas]

        fig, ax1 = plt.subplots()
        ax1.bar(fechas, ingresos_vals, color='skyblue', label="Ingresos")
        ax1.set_ylabel("Ingresos")
        ax1.tick_params(axis='x', rotation=45)

        ax2 = ax1.twinx()
        ax2.plot(fechas, huespedes_vals, color='red', marker='o', label="Huéspedes")
        ax2.set_ylabel("Huéspedes")

        plt.title("Ingresos vs Huéspedes por día")
        fig.tight_layout()
        plt.show()
    else:
        print("No hay fechas para generar gráfica combinada (8).")


In [ ]:
menu()